In [10]:
def negator(s):
    s = s.replace("n't", " not").replace(".", " . ")
    should_negate_word = False
    text = ''
    words = s.split()
    for word in words:
        if word == 'not' or word == 'no' or word == 'n\'t':
            should_negate_word = not should_negate_word
        elif word == '.':
            should_negate_word = False
        elif should_negate_word:
            text += 'not_' + word + ' '
        else:
            text += word + ' '
    return text

In [11]:
from nltk.stem import PorterStemmer
from nltk.tokenize import SpaceTokenizer
import re
import csv

stopwords = open('stopwords.csv').readlines()[0].rstrip().split(', ')

tokenizer = SpaceTokenizer()
stemmer = PorterStemmer()

pipeline = [lambda s: re.sub('<[^>]*>', ' ', s),
            lambda s: re.sub("[^.'\w\s]", '', s),
            lambda s: re.sub('[\d]', '', s),
            lambda s: s.lower(),
            lambda s: ' '.join(filter(lambda s: not (s in stopwords), tokenizer.tokenize(s))),
            #lambda s: ' '.join(map(lambda t: stemmer.stem(t), tokenizer.tokenize(s))),
            lambda s: negator(s)
           ]

def preprocess_text(text, pipeline=pipeline):
    if len(pipeline) == 0:
        return text
    else:
        return preprocess_text(pipeline[0](text), pipeline[1:])

In [12]:
def preprocess_review(r, test=False):
    if (test):
        SUMMARY_INDEX = 7
        TEXT_INDEX = 8  
    else:
        SUMMARY_INDEX = 8
        TEXT_INDEX = 9
    
    r[SUMMARY_INDEX] = preprocess_text(r[SUMMARY_INDEX])
    r[TEXT_INDEX] = preprocess_text(r[TEXT_INDEX])
    return r

In [13]:
import csv
import sys

reload(sys)
sys.setdefaultencoding('utf-8')

def parallel_preprocess(input_file, output_file, test=False):
    # Load csv
    reviewsRDD = sc.textFile(input_file, use_unicode=False)
    reviewsRDD = reviewsRDD.mapPartitions(lambda x: csv.reader(x))

    # Save header
    header = reviewsRDD.first()

    # Extract header and preprocess
    reviewsRDD = reviewsRDD.filter(lambda x: x != header)\
                           .map(lambda x: preprocess_review(x, test))
    ppreviews = reviewsRDD.collect()

    # Insert header
    ppreviews.insert(0, header)

    # Save csv
    with open(output_file, 'wb') as f:
        writer = csv.writer(f)
        writer.writerows(ppreviews)

In [16]:
# Add test=True to preprocess test files
#parallel_preprocess('../data/test.csv', '../data/sparktest.csv', test=True)

In [24]:
# TODO add this functions to the pipeline
# Run this if you want to join words in shingles
def ngram(r, size, test):
    if (test):
        SUMMARY_INDEX = 7
        TEXT_INDEX = 8  
    else:
        SUMMARY_INDEX = 8
        TEXT_INDEX = 9
    words = r[TEXT_INDEX].split()
    shingles = [words[i:i+size] for i in xrange(len(words) - size + 1)]
    shingles = ["_".join(i) for i in shingles]
    r[TEXT_INDEX] = " ".join(shingles)
    return r

def create_shingles():
    reviewsRDD = sc.textFile('../data/sparktrain.csv', use_unicode=False)
    reviewsRDD = reviewsRDD.mapPartitions(lambda x: csv.reader(x))
    # Save header
    header = reviewsRDD.first()
    # Exctract header
    reviewsRDD = reviewsRDD.filter(lambda x: x != header)
    reviewsRDD = reviewsRDD.map(lambda x: ngram(x, 2, False)) 

[['492346',
  'B001VNP0Y6',
  'AEZRVY3CV52UZ',
  'Corey Wright "Democratus"',
  '3',
  '3',
  '5',
  '1320537600',
  'needed ',
  "very_timely timely_manner manner_they're they're_very very_large large_green green_seems seems_inordinate inordinate_amount amount_seeds seeds_come come_out out_weird weird_beige beige_color color_still still_tastes tastes_great great_much much_better better_deal deal_indian indian_markets markets_town town_cheers cheers_frontier"],
 ['343811',
  'B008J1HO4C',
  'A18VZYSLOUO060',
  'D. Simpson "Frugal SOB"',
  '2',
  '2',
  '5',
  '1204329600',
  'excellent oatmeal ',
  "mccann's_steel steel_cut cut_oatmeal oatmeal_perfect perfect_breakfast breakfast_people people_hurry hurry_put put_cup cup_oats oats_little little_crockpot crockpot_cups cups_boiling boiling_water water_before before_go go_bed bed_hour hour_before before_up up_crockpot crockpot_starts starts_high high_timer timer_out out_shower shower_oatmeal oatmeal_ready ready_go go_mix mix_blueberries bl

In [22]:
a = "very timely manner they're very large green seems inordinate amount seeds come out weird beige color still tastes great much better deal indian markets town cheers frontier "
words = a.split()
words
size = 2
b = [words[i:i+size] for i in xrange(len(words) - size + 1)]
c = ["_".join(i) for i in b]
" ".join(c)

"very_timely timely_manner manner_they're they're_very very_large large_green green_seems seems_inordinate inordinate_amount amount_seeds seeds_come come_out out_weird weird_beige beige_color color_still still_tastes tastes_great great_much much_better better_deal deal_indian indian_markets markets_town town_cheers cheers_frontier"